# ReaDDy intro I: Particles, diffusion and potentials
In this ipython notebook we will cover:
- principle workflow of readdy: `system` and `simulation`
- adding particle species to `system`
- adding potentials to `system`
- spatial layout of simulation box and box potentials [link to simulation box setup in documentation](link)
- adding particle instances at given positions to `simulation`
- convert trajectory output to VMD viewable format


The __principal workflow__ of ReaDDy is
1. Create and configure a `readdy.ReactionDiffusionSystem`, i.e. particle species, potentials, reactions, topologies
2. Get a `Simulation` object and configure it, i.e. output file, observables, initial particle positions
3. Run the `Simulation`, by choosing a time step size and the number of integration steps
4. Analyze results

Keep in mind:
- When in doubt, refer to the online documentation https://readdy.github.io !
- Make use of tab-completion in notebooks !

In [ ]:
import os
import numpy as np
import readdy
print(readdy.__version__)

## System
The first step is to create a `ReactionDiffusionSystem`. The system defines the parameters that will not change during the course of a simulation.

In [ ]:
system = readdy.ReactionDiffusionSystem(box_size=[10.,10.,10.])

that has various properties

In [ ]:
system.box_size

In [ ]:
system.temperature

In [ ]:
system.kbt

In [ ]:
system.periodic_boundary_conditions

In [ ]:
system.length_unit, system.time_unit, system.energy_unit

We can add particle species (types)

In [ ]:
system.add_species("A", diffusion_constant=1.0)

## Simulation
From this system, we can create a `Simulation` object, which runs on a specific kernel

In [ ]:
simulation = system.simulation(kernel="SingleCPU")

In [ ]:
simulation.output_file = "./oneparticle.h5"
simulation.record_trajectory()

Set the initial position in 3D coordinates of a single particle

In [ ]:
simulation.add_particle("A", [0.,0.,0.])

Run the simulation

In [ ]:
if os.path.exists(simulation.output_file):
    os.remove(simulation.output_file)

simulation.run(n_steps=1000, timestep=0.01)

Look at results. The `!` prefix in a cell forwards the command to the (bash) shell.

In [ ]:
!ls | grep oneparticle

Read the trajectory file, and convert the results to a VMD compatible format.

In [ ]:
traj = readdy.Trajectory(filename="oneparticle.h5")

In [ ]:
traj.convert_to_xyz()

In [ ]:
!ls | grep oneparticle

In [ ]:
!vmd -e oneparticle.h5.xyz.tcl > /dev/null

## Two particles

In [ ]:
system = readdy.ReactionDiffusionSystem([10.,10.,10.], unit_system=None)
system.add_species("A", diffusion_constant=1.0)
system.add_species("B", diffusion_constant=0.1)

In [ ]:
simulation = system.simulation("SingleCPU")
simulation.output_file = "twospecies.h5"
simulation.record_trajectory()
simulation.add_particle("A", [0.,0.,0.])
simulation.add_particle("B", [1.,1.,1.])

In [ ]:
if os.path.exists(simulation.output_file):
    os.remove(simulation.output_file)

simulation.run(1000, 0.01)

In [ ]:
!ls | grep twospecies

In [ ]:
traj = readdy.Trajectory("twospecies.h5")
traj.convert_to_xyz()

In [ ]:
!vmd -e twospecies.h5.xyz.tcl > /dev/null

## Many particles

In [ ]:
system = readdy.ReactionDiffusionSystem([10.,10.,10.], unit_system=None)
system.add_species("A", diffusion_constant=1.0)
system.add_species("B", diffusion_constant=0.1)

In [ ]:
simulation = system.simulation("SingleCPU")
simulation.output_file = "manyparticles.h5"
simulation.record_trajectory()
n_particles = 100
for i in range(n_particles):
    simulation.add_particle("A", np.random.uniform(size=3)*10. - 5.)
    simulation.add_particle("B", np.random.uniform(size=3)*10. - 5.)

In [ ]:
if os.path.exists(simulation.output_file):
    os.remove(simulation.output_file)

simulation.run(1000, 0.01)

In [ ]:
!ls | grep manyparticles

In [ ]:
traj = readdy.Trajectory("manyparticles.h5")
traj.convert_to_xyz()

In [ ]:
!vmd -e manyparticles.h5.xyz.tcl > /dev/null

What happens at the boundary?

### Simulation box, box potential
see https://readdy.github.io/workshop_sessions.html#cookbook
and https://readdy.github.io/system.html#box-potential

What happens if system is non-periodic?

In [ ]:
system = readdy.ReactionDiffusionSystem(
    [10.,10.,10.], 
    periodic_boundary_conditions=[False, False, False],
    unit_system=None
)
system.add_species("A", 1.)

In [ ]:
if True:
    system.potentials.add_box("A", force_constant=20., origin=[-3.,-3.,-3.], extent=[6.,6.,6.])

In [ ]:
simulation = system.simulation("SingleCPU")
simulation.output_file = "box.h5"
simulation.record_trajectory()
simulation.add_particles("A", np.zeros((40,3)))

if os.path.exists(simulation.output_file):
    os.remove(simulation.output_file)
simulation.run(10000, 0.01)

In [ ]:
traj = readdy.Trajectory("box.h5")
traj.convert_to_xyz(tcl_with_grid=True)

In [ ]:
!vmd -e box.h5.xyz.tcl > /dev/null

Note that all particles must always be in the simulation box. If the system is non-periodic, you should use a box potential to contain the particles inside.
The following image shows the relation of the simulation box and the box potential's _origin_ and _extent_
![](https://readdy.github.io/assets/box_potential_within.png)

## Confine particles to 2D
You can use a box potential with a very small _extent_ in one dimension to confine particles to a very thin 3D sheet.

In [ ]:
system = readdy.ReactionDiffusionSystem(
    [10.,10.,10.], 
    periodic_boundary_conditions=[False, False, False],
    unit_system=None
)
system.add_species("A", 1.)
system.potentials.add_box("A", force_constant=20., origin=[-3.,-3.,0.], extent=[6.,6.,0.01])

In [ ]:
simulation = system.simulation("SingleCPU")
simulation.output_file = "box2d.h5"
simulation.record_trajectory()
simulation.add_particles("A", np.zeros((40,3)))

if os.path.exists(simulation.output_file):
    os.remove(simulation.output_file)
simulation.run(10000, 0.01)

In [ ]:
traj = readdy.Trajectory("box2d.h5")
traj.convert_to_xyz(tcl_with_grid=True)

In [ ]:
!vmd -e box2d.h5.xyz.tcl > /dev/null